# OpenStreetMap Data Wrangling with MongoDB
## Jenni Sanders

#### Map Area: Tulsa, OK, United States

### Problems Encountered

* Abbreviated and misspelled street names
  * Ave. instead of Avenue
  * Bouleavard instead of Boulevard
* Second-level Tags
  * payment:electronic_purses
  * addr:street
* Phone Number Formatting

Additionally, the most common top-level tag found in the data was 'tiger'.  I considered this unusual, did some research, and these tags are part of the TIGER import of the United States (http://wiki.openstreetmap.org/wiki/Key:tiger).

#### Abbreviated and Misspelled Street Names

There were a lot of street names where the type of street, e.g. Avenue or Boulevard, was either abbreviated or mispelled.  I've corrected those where possible.  When a street type is missing, e.g. 'South Memorial' instead of 'South Memorial Drive', I haven't assumed what street type should be present.

#### Second-level Tags
Out of the 421160 tags in the data, 196217 or just under half had second-level tags.  Because they are in the original data like tag1:tag2, I decided to make these so that the second-level tags, e.g. electronic_purses, would be a subset of payment.

#### Phone Number Formatting
Phone numbers were formatted in multiple ways, including +1 prefixes before the number, parenthesis inside the number, and occasionally periods within the number.
```python
numeric = re.compile(r'^([0-9])*$')
numericdashes = re.compile(r'^([0-9]|\-)*$')

def phone_number_format(element, keys):
    other_formatted_numbers = []
    problem_formatted_numbers = []
    
    if element.tag == "tag" and element.attrib['k'] == "phone":
        if numeric.match(element.attrib['v']):
            keys['numeric'] += 1
        if numericdashes.match(element.attrib['v']):
            keys['numericdashes'] += 1
        elif element.attrib['v'][0] == '+' and numericdashes.match(element.attrib['v'][1:]):
            keys['beginwplus'] += 1
        elif problemchars.search(element.attrib['v']) is not None:
            keys['problemchars'] += 1
            problem_formatted_numbers.append(element.attrib['v'])
        else:
            keys['other'] += 1
            other_formatted_numbers.append(element.attrib['v'])
    
#     if len(other_formatted_numbers) > 0:
#         print other_formatted_numbers
    
#     if len(problem_formatted_numbers) > 0:
#         print problem_formatted_numbers
    
    return keys


def process_map_phonekeys(filename):
    keys = {"numeric": 0, "numericdashes": 0, "beginwplus":0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = phone_number_format(element, keys)

    return keys

phonekeys = process_map_phonekeys(OSMFILE)
pprint.pprint(phonekeys)```

I used the phonenumbers library to normalize these to the US national number format.
```python
import phonenumbers as pn

def tidy_phone(phonenumber):
    return pn.format_number(pn.parse(phonenumber, 'US'), pn.PhoneNumberFormat.NATIONAL)```

### Overview of Data

The extract was a custom extract from Mapzen with a key of odes-agCKTKY.  This extract consists of 86,113 KB.  When exported to JSON, this becomes a 133,866 KB file.  The storage size in Mongo is 33734656, with 396501 objects.  

These objects are made of 339454 nodes:  
```python
db.OSM.find( { "type" : "node" }).count()```  

and 57047 ways:  
```python
db.OSM.find( { "type" : "way" }).count()```

contributed by 371 users.
```python
def count_users_pipeline():
    pipeline = [
        { "$match" : { "user" : { "$exists" : True }}}, ##only count changes with a username attached
        { "$group" : { "_id" : "$user", "count": { "$sum" : 1 }}},
        { "$group" : { "_id" : "unique users", "count" : { "$sum" : 1 }}}
    ]
    return pipeline
    
pipeline = count_users_pipeline()
result = aggregate(db, pipeline)
pprint.pprint(result[0])```

The most active user in the dataset is Paul Johnson, with 133,819 changes.
```python
def user_pipeline():
    pipeline = [
        { "$match" : { "user" : { "$exists" : True }}}, ##only count changes with a username attached
        { "$group" : { "_id" : "$user", "count": { "$sum" : 1 }}},
        { "$sort" : { "count" : -1 }},
        { "$limit" : 1 }
    ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.OSM.aggregate(pipeline)]

pipeline = user_pipeline()
result = aggregate(db, pipeline)
pprint.pprint(result[0])```

This name sounded familiar, so I did some searching and in 2014 this person had over a million nodes in total changes. https://codefortulsa.org/2014/01/19/tulsas-first-openstreetmap-editathon/

### Additional Ideas

It would be interesting to reconcile this data set with the USPS data set to determine which addresses in the set are valid, and potentially reconcile incomplete or incorrect addresses to improve data accuracy and reliability.  For example, the following address is in the data set: 

```python
for node in db.OSM.find({'address.street' : {'$regex' : 'Tulsa Promenade'}}):
    pprint.pprint(node)```
    
148 Tulsa Promenade, should be:

   TULSA PROMENADE  
   4107 S YALE AVE  
   STE 148  
   TULSA OK 74135-6015  
   
Data could be fed through a package like pyusps or postal-address to improve accuracy and normalize addresses.  However, given the manual nature of many of the Open Street Map entries, human verification would be recommended for at least a sample subset of this data to validate accuracy.

#### Additional Exploration  
There's a running joke in Tulsa that there are an equal number of bars and churches.  Analyzing the data set, bars aren't even in the top five amenities:
```python
def count_amenities_pipeline():
    pipeline = [
        { "$match" : {"amenity" : { "$exists" : True }, "type" : "node" }}, #only nodes with an amenity tag
        { "$group" : { "_id" : "$amenity", "count": {"$sum" : 1 }}},
        { "$sort" : { "count" : -1 }}
    ]
    return pipeline

pipeline = count_amenities_pipeline()
result = aggregate(db, pipeline)
pprint.pprint(result[0:5])```

The top five are:
* Place of Worship, 554
* School, 140
* Fast Food, 92
* Restaurant, 76
* Fountain, 67

Even assuming some self-selection, that bar number looks pretty low.  I looked through all the names of amenities to determine what types of ameneties appeared to be places where the primary source of income would be alcohol sales.  Using that list, I looked at just those establishments and places of worship to determine the numbers between the two:
``` python
def count_certain_amenities_pipeline():
    pipeline = [
        { "$match" : {"amenity" : { "$in" : ["place_of_worship", "bar", "pub", "liquor", "nightclub"]} , "type" : "node" }}, #only the nodes we're interested in
        { "$group" : { "_id" : "$amenity", "count": {"$sum" : 1 }}},
        { "$sort" : { "count" : -1 }}
    ]
    return pipeline

pipeline = count_certain_amenities_pipeline()
result = aggregate(db, pipeline)
pprint.pprint(result)```

* Place of Worship, 554
* Bar, 9
* Pub, 8
* Liquor, 2
* Nightclub, 1

Given the above totals, it would be far easier to use this data set to find a place of worship than a watering hole.

### Conclusion
The Tulsa data set seems fairly clean, especially considering there were _no_ problematic characters in the tags.  It could use some address scrubbing, as is expected with a manually created data set.  Whenever people touch data, it's likely they will use shortcuts, skip unknown data, and make typos in the data.  For example, I've frequently found 111111111 or 999999999 listed as social security numbers in other data sets.  A bot to rectify address data in this set with a known accurate data source would be a large step forward in data accuracy in this set.

### References:

7.2. re — Regular expression operations¶. (n.d.). Retrieved January 02, 2017, from https://docs.python.org/2/library/re.html#simulating-scanf  
Delete everything in a MongoDB database. (n.d.). Retrieved January 02, 2017, from http://stackoverflow.com/questions/3366397/delete-everything-in-a-mongodb-database  
Efficient way to remove keys with empty values from a dict. (n.d.). Retrieved January 02, 2017, from http://stackoverflow.com/questions/12118695/efficient-way-to-remove-keys-with-empty-values-from-a-dict  
Find or Query Data with the mongo Shell¶. (n.d.). Retrieved January 02, 2017, from https://docs.mongodb.com/getting-started/shell/query/  
How do I query mongodb with "like"? (n.d.). Retrieved January 02, 2017, from http://stackoverflow.com/questions/3305561/how-do-i-query-mongodb-with-like  
Install MongoDB Community Edition on Windows¶. (n.d.). Retrieved January 02, 2017, from https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/  
MongoDB select count(distinct x) on an indexed column - count unique results for large data sets. (n.d.). Retrieved January 02, 2017, from http://stackoverflow.com/questions/11782566/mongodb-select-countdistinct-x-on-an-indexed-column-count-unique-results-for  
Pymongo method of getting statistics for collection byte usage? (n.d.). Retrieved January 02, 2017, from http://stackoverflow.com/questions/18836064/pymongo-method-of-getting-statistics-for-collection-byte-usage  
Python Regex Cheatsheet. (n.d.). Retrieved January 2, 2017, from https://www.debuggex.com/cheatsheet/regex/python  
B. (2014, January 19). Tulsa’s First OpenStreetMap Editathon. Retrieved January 02, 2017, from https://codefortulsa.org/2014/01/19/tulsas-first-openstreetmap-editathon/  
Udacity | Free Online Courses & Nanodegree Programs - Udacity. (n.d.). Retrieved January 02, 2017, from https://discussions.udacity.com/t/how-to-set-up-mongodb-locally-windows/185014/2